# This code solves the missionaries and cannibals problem

In [10]:
class state_space:
    def show(self):
        
        attributes = vars(self)
        print(', '.join("%s: %s" % item for item in attributes.items()))

    class state:    
        def __init__(self, cannibals = 3, missionaries = 3, boat = 'L'):
            #state vector
            self.cannibals_left_bank = cannibals
            self.missionaries_left_bank = missionaries
            self.boat = boat
            self.name = str(self.cannibals_left_bank) + \
                 str(self.missionaries_left_bank) + \
                 str(self.boat)
            
            #validity check
            self.valid = None
            
            cannibals_right_bank = 3 - self.cannibals_left_bank
            missionaries_right_bank = 3 - self.missionaries_left_bank
    
            
            if (self.cannibals_left_bank > self.missionaries_left_bank) & (self.missionaries_left_bank > 0) :
                self.valid = False
                
            elif (cannibals_right_bank > missionaries_right_bank) & (missionaries_right_bank > 0):
                self.valid = False
                
            else:
                self.valid = True
            
            #check if final state
            self.final_state = None
            
            if (self.cannibals_left_bank == 0) & (self.missionaries_left_bank == 0 ):
                self.final_state = True
                
            else:
                self.final_state = False
            
            
        def show(self):
            
            print('c', self.cannibals_left_bank)
            print('m', self.missionaries_left_bank)
    
    class state_tree:

        class node:
            def __init__(self, parent, depth, state, incoming_edge = None):
                self.parent = parent
                self.children = [None]
                self.state = state
                self.depth = depth
                self.incoming_edge = incoming_edge
                self.outgoing_edges = []
                
            def show(self):
                print('state', self.state)
                print('depth', self.depth)
                print('parent', self.parent)
                print('incoming edge', self.incoming_edge)
                
                #attributes = vars(self)
                #print(', '.join("%s: %s" % item for item in attributes.items()))
        
        def get_level(self, target_depth):
            level = []
            
            for node in self.nodes:
                
                if node.depth == target_depth:
                    level.append(node)
                    
            return level
        
        def __init__(self, root):
            self.nodes = [self.node(None, 0, root, None)]
            self.edges = [{}]
            

            
    class state_registry():
        #dict of states, key is 2 digit string "XY". X is cannibals left bank. Y is missionaries left bank.
        
        registry = {}
                
    def state_change(self, prior_state, p0, p1, p2, direction): 
        #Applies move to state, returns outcome state
        
        delta_missionaries = 0
        delta_cannibals = 0 
        passenger_list = [p0, p1, p2]
        #print(delta_cannibals, delta_missionaries,passenger_list)
        
        if(direction == 'R') & (prior_state.boat == "L"):
            
            for item in passenger_list:
                
                if item == 0:
                    delta_missionaries = delta_missionaries - 1
                    
                    if (prior_state.missionaries_left_bank + delta_missionaries < 0):
                        return None
                    
                elif item == 1:
                    delta_cannibals = delta_cannibals - 1
                    
                    if (prior_state.cannibals_left_bank + delta_cannibals < 0):
                        return None
                    
                elif item is None:
                    pass

        elif(direction == 'L') & (prior_state.boat == "R"):
            
            for item in passenger_list:
                
                if item == 0:
                    delta_missionaries = delta_missionaries + 1
                    
                    if prior_state.missionaries_left_bank + delta_missionaries > 3:
                        return None
                    
                elif item == 1:  
                    delta_cannibals = delta_cannibals + 1
                    
                    if prior_state.cannibals_left_bank + delta_cannibals > 3:
                        return None
                elif item is None:
                    pass
        else:
            
            return None

        new_state_inputs = [prior_state.cannibals_left_bank + delta_cannibals, 
                            prior_state.missionaries_left_bank + delta_missionaries]
        #print(new_state_inputs)
        new_state = self.state(*new_state_inputs, direction)
        
        return new_state

    def get_edges(self, current_state): 
        #Apply all possible moves to a state object. Return dictionary. Dict key is string format
        #"0NoneR". 
        leaf_dict = {}
        
        for move in self.possible_moves:
            #print('move', move, '\n state')
            next_state = self.state_change(current_state, *move)
            
            if next_state is not None:
                #next_state.show()
                leaf_dict.update({str(move):next_state})

        return leaf_dict 

    def resolve_state_space(self):
    #populates state_registry.registry dict state names are string "XY" X = n cannibals 
    #Y = n missionaries
    
        for cannibals in range(4):
            
            for missionaries in range(4):
                
                for boat in self.directions:
                    state = self.state(cannibals,missionaries, boat)
                    self.state_registry.registry.update({state.name : state})
                    
    def resolve_graph(self):
        #determine edges of graph, where states are vertices. return dict of dicts.
        #key is state, value is dict where key is move, secondary value is new state
        edges = {}
        
        for state in self.state_registry.registry:
            
            if self.state_registry.registry.get(state).valid == True:
                name = self.state_registry.registry.get(state).name
                edges.update({name : self.get_edges(self.state_registry.registry.get(state))})
            #print(state)
            #print(self.state_registry.registry.get(state))
            
        return edges
                    
    def resolve_tree(self, depth):        
    #populate tree with nodes and vertices until depth is reached
        limit = range(depth)
        tree = self.state_tree(self.root)
        
        for counter in limit:
            print('counter',counter)
            #next_level = []
            current_level = tree.get_level(counter)
            print(current_level)
            level_state_registry = []
            
            for this_node in current_level:
            
                if this_node.state.valid:
                    sub_graph = self.graph[str(this_node.state.name)]
                    #print(sub_graph)
                    
                    for key in sub_graph:
                        #ref--class node:
                        #def __init__(self, parent, depth, state, 
                        #incoming_edge = None):
                        
                        new_node = tree.node(this_node, counter +1 \
                        , sub_graph.get(key), key)

                        #print('new node', new_node)
                        if new_node.state.name not in level_state_registry:
                            #print(new_node.state.name)
                            level_state_registry.append(new_node.state.name)
                            tree.nodes.append(new_node)
                            
                else:
                    print('inval')
        return tree
                    
    def __init__(self):

        self.passenger_types = {0: 'missionary', 1:'cannibal', None : 'empty'}
        self.directions = ['L', 'R']
        self.root = self.state(3,3)
              
        def enumerate_possible_moves(self):
        #Create list of all possible moves, without consideration of board state. 
        #Returns list of moves, which are also lists. Does not produce duplicate moves.
            passenger_count_registry = []
            possible_moves = []
            skips = []
            for direction in self.directions:
                skips = []
                passenger_count_registry = []
                for seat0 in self.passenger_types:

                    for seat1 in self.passenger_types:
                        #print(item0, item1)
                        #if (seat0 is None) & (seat1 is None):
                        #    pass
                        for seat2 in self.passenger_types:
                            missionaries_boat = sum([(seat0 == 0),(seat1 == 0), (seat2 == 0)])
                            cannibals_boat = sum([(seat0 == 1), (seat1 == 1), (seat2 == 1)])
                            if([missionaries_boat,cannibals_boat] not in passenger_count_registry):
                                passenger_count_registry.append([missionaries_boat, cannibals_boat])
                                if (seat0 is None) & (seat1 is None) & (seat2 is None):
                                    pass

                                else:                            
                                    possible_moves.append([seat0, seat1, seat2, direction])

#                    print('seat0',seat0, 'seat1', seat1)
                    skips.append(seat0)
#                    print('skips', skips)
            return possible_moves

        self.possible_moves = enumerate_possible_moves(self)

        self.state_space = self.resolve_state_space()
        
        self.graph = self.resolve_graph()


In [11]:
S = state_space()
#

In [12]:

S.possible_moves

[[0, 0, 0, 'L'],
 [0, 0, 1, 'L'],
 [0, 0, None, 'L'],
 [0, 1, 1, 'L'],
 [0, 1, None, 'L'],
 [0, None, None, 'L'],
 [1, 1, 1, 'L'],
 [1, 1, None, 'L'],
 [1, None, None, 'L'],
 [0, 0, 0, 'R'],
 [0, 0, 1, 'R'],
 [0, 0, None, 'R'],
 [0, 1, 1, 'R'],
 [0, 1, None, 'R'],
 [0, None, None, 'R'],
 [1, 1, 1, 'R'],
 [1, 1, None, 'R'],
 [1, None, None, 'R']]

In [13]:
S.state_registry.registry

{'00L': <__main__.state_space.state at 0x7fe414492b70>,
 '00R': <__main__.state_space.state at 0x7fe414472e10>,
 '01L': <__main__.state_space.state at 0x7fe414472e48>,
 '01R': <__main__.state_space.state at 0x7fe414472860>,
 '02L': <__main__.state_space.state at 0x7fe414472d30>,
 '02R': <__main__.state_space.state at 0x7fe414472908>,
 '03L': <__main__.state_space.state at 0x7fe414472a20>,
 '03R': <__main__.state_space.state at 0x7fe414472f98>,
 '10L': <__main__.state_space.state at 0x7fe414472630>,
 '10R': <__main__.state_space.state at 0x7fe414472f28>,
 '11L': <__main__.state_space.state at 0x7fe414472f60>,
 '11R': <__main__.state_space.state at 0x7fe4144d7080>,
 '12L': <__main__.state_space.state at 0x7fe4144d7828>,
 '12R': <__main__.state_space.state at 0x7fe4144d7c18>,
 '13L': <__main__.state_space.state at 0x7fe4144d7e80>,
 '13R': <__main__.state_space.state at 0x7fe4144d7978>,
 '20L': <__main__.state_space.state at 0x7fe4144d7f28>,
 '20R': <__main__.state_space.state at 0x7fe4144

In [14]:
S.graph = S.resolve_graph()
print(S.graph)

{'00L': {}, '00R': {"[0, 0, 0, 'L']": <__main__.state_space.state object at 0x7fe4144522e8>, "[0, 0, 1, 'L']": <__main__.state_space.state object at 0x7fe414452518>, "[0, 0, None, 'L']": <__main__.state_space.state object at 0x7fe4144521d0>, "[0, 1, 1, 'L']": <__main__.state_space.state object at 0x7fe4144524e0>, "[0, 1, None, 'L']": <__main__.state_space.state object at 0x7fe414452080>, "[0, None, None, 'L']": <__main__.state_space.state object at 0x7fe414452208>, "[1, 1, 1, 'L']": <__main__.state_space.state object at 0x7fe414452470>, "[1, 1, None, 'L']": <__main__.state_space.state object at 0x7fe41443ceb8>, "[1, None, None, 'L']": <__main__.state_space.state object at 0x7fe41443cbe0>}, '03L': {"[0, 0, 0, 'R']": <__main__.state_space.state object at 0x7fe41443c7b8>, "[0, 0, None, 'R']": <__main__.state_space.state object at 0x7fe41443c0f0>, "[0, None, None, 'R']": <__main__.state_space.state object at 0x7fe41443c358>}, '03R': {"[1, 1, 1, 'L']": <__main__.state_space.state object at 

In [15]:
tree = S.resolve_tree(11)

counter 0
counter 1
[<__main__.state_space.state_tree.node object at 0x7fe4143e9f60>, <__main__.state_space.state_tree.node object at 0x7fe4143e9f98>, <__main__.state_space.state_tree.node object at 0x7fe4143e9ef0>, <__main__.state_space.state_tree.node object at 0x7fe4143e9f28>, <__main__.state_space.state_tree.node object at 0x7fe4143e9e80>, <__main__.state_space.state_tree.node object at 0x7fe4143e9eb8>, <__main__.state_space.state_tree.node object at 0x7fe4143e9e10>, <__main__.state_space.state_tree.node object at 0x7fe4143e9e48>, <__main__.state_space.state_tree.node object at 0x7fe4143e9da0>]
inval
inval
inval
inval
counter 2
[<__main__.state_space.state_tree.node object at 0x7fe4143e9d30>, <__main__.state_space.state_tree.node object at 0x7fe4143e9d68>, <__main__.state_space.state_tree.node object at 0x7fe4143e9cc0>, <__main__.state_space.state_tree.node object at 0x7fe4143e9c50>, <__main__.state_space.state_tree.node object at 0x7fe4143e9c88>]
inval
inval
counter 3
[<__main__.s

In [16]:
print(S.possible_moves)

[[0, 0, 0, 'L'], [0, 0, 1, 'L'], [0, 0, None, 'L'], [0, 1, 1, 'L'], [0, 1, None, 'L'], [0, None, None, 'L'], [1, 1, 1, 'L'], [1, 1, None, 'L'], [1, None, None, 'L'], [0, 0, 0, 'R'], [0, 0, 1, 'R'], [0, 0, None, 'R'], [0, 1, 1, 'R'], [0, 1, None, 'R'], [0, None, None, 'R'], [1, 1, 1, 'R'], [1, 1, None, 'R'], [1, None, None, 'R']]


In [17]:
#breadth first 
#traverses tree in order of creation

for node in tree.nodes:
    #if node.state.valid:
        print ('name', node.state.name, ', depth', node.depth, ', incoming edge', node.incoming_edge)
        if node.state.final_state == True:
            soln = node
            print('Solution found after ' ,node.depth , 'steps.')
            print(node.depth)
            break


name 33L , depth 0 , incoming edge None
name 30R , depth 1 , incoming edge [0, 0, 0, 'R']
name 21R , depth 1 , incoming edge [0, 0, 1, 'R']
name 31R , depth 1 , incoming edge [0, 0, None, 'R']
name 12R , depth 1 , incoming edge [0, 1, 1, 'R']
name 22R , depth 1 , incoming edge [0, 1, None, 'R']
name 32R , depth 1 , incoming edge [0, None, None, 'R']
name 03R , depth 1 , incoming edge [1, 1, 1, 'R']
name 13R , depth 1 , incoming edge [1, 1, None, 'R']
name 23R , depth 1 , incoming edge [1, None, None, 'R']
name 33L , depth 2 , incoming edge [0, 0, 0, 'L']
name 32L , depth 2 , incoming edge [0, 0, None, 'L']
name 31L , depth 2 , incoming edge [0, None, None, 'L']
name 23L , depth 2 , incoming edge [0, None, None, 'L']
name 13L , depth 2 , incoming edge [1, None, None, 'L']
name 30R , depth 3 , incoming edge [0, 0, 0, 'R']
name 21R , depth 3 , incoming edge [0, 0, 1, 'R']
name 31R , depth 3 , incoming edge [0, 0, None, 'R']
name 12R , depth 3 , incoming edge [0, 1, 1, 'R']
name 22R , dept